In [ ]:
# Attention Mechanism
# how one word in a sequence is related to other
# ex : cat sit ion the mat
# word embedding  --->  understnad menaing
# text to int conversion (ml) - NLP (Nartural Lang Proceing)
# 3 mainthod to (text to int conversion)   -  word embedding ,  one-hot hot,  interter encondin
# one hot encoding  - ex: cat sit ion the mat
 #  create a unique words of thedata [ cat,  sit , on, the, mat]
 # create a sperse vector for each vocab  cat [1,0,0,0,0] , sit [0,1,0,0,0]
 #diasvdatge - cannt unsetand the semantic / content ,  large spare vector

# inter encoding - [ 1,  2 ,3, 4, 5] ,  dense vector, most compact ,  no context, semantic

# word embdding - have trainable paremeter (vectors) -   embeddinglayer(5, 2)
# cat [0.1, 0.5],  sit [[0.1, 0.5]] , on [0.1, 0.5], the [0.1, 0.5], mat[0.1, 0.5]
# train model : 
# its create a vector space for all the vocabulary
 


In [6]:
# word embdding
import os
import tensorflow as tf

# prepare data
train_dir = os.path.join("aclImdb_v1_extracted/aclImdb/test")

os.listdir(train_dir)


['neg', 'urls_pos.txt', 'urls_neg.txt', 'pos', 'labeledBow.feat']

In [10]:
train_ds = tf.keras.utils.text_dataset_from_directory(train_dir , validation_split=0.2, subset="training" ,batch_size=32 ,seed=42)
valid_ds = tf.keras.utils.text_dataset_from_directory(train_dir, validation_split=0.2, subset="validation", batch_size=32, seed=42)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [15]:
_ , label = next(iter(train_ds))
label

<tf.Tensor: shape=(32,), dtype=int32, numpy=
array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int32)>

In [17]:
# what is cache and prefetch ?  chcek - small to medium , that can fit inside you computer ram or harddisk
# confihure datset for performance , prefetch - one batch is training in gpus, other prepare and setup in cpu , AUTOTUNE?
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size = AUTOTUNE)

valid_ds = valid_ds.cache().prefetch(buffer_size=  AUTOTUNE)


In [28]:
# model  = nn.Suwnetion([embedding,  dense])
emdeb_layer = tf.keras.layers.Embedding(5, 2) 

a = tf.constant([1, 2, 3, 4, 0, 3 ]) # shape(3,)

emdeb_layer(a) 

<tf.Tensor: shape=(6, 2), dtype=float32, numpy=
array([[ 0.00576295,  0.01189106],
       [ 0.04488179,  0.02726892],
       [-0.03844514, -0.02120103],
       [ 0.00848771,  0.0305177 ],
       [-0.03398626, -0.0248061 ],
       [-0.03844514, -0.02120103]], dtype=float32)>

In [37]:
import string
import re
def standardizer(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


In [54]:
vectorized_layer = tf.keras.layers.TextVectorization(standardize = standardizer,
                                                      max_tokens = 10000,
                                                      output_sequence_length = 100,
                                                      output_mode='int',

                                                      )



In [55]:
vectorized_layer.get_vocabulary()

['', '[UNK]']

In [57]:
text_ds = train_ds.map(lambda x, y : x)

vectorized_layer.adapt(text_ds)

2025-05-24 16:07:32.733973: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [58]:
vectorized_layer.get_vocabulary()[100]

np.str_('any')

In [129]:
vocab_size = 10000
embedding_dim = 8
model = tf.keras.Sequential([
    vectorized_layer,
    tf.keras.layers.Embedding(vocab_size, embedding_dim),  # 32,100,8
    tf.keras.layers.GlobalAveragePooling1D(), #32,8
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
    
])

In [130]:
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()


model.compile(loss=loss_fn, optimizer=optimizer, metrics=["accuracy"])

In [131]:
tensorboard_callbacks = [tf.keras.callbacks.TensorBoard(log_dir="my_logs"),
                         tf.keras.callbacks.EarlyStopping(patience=6,  monitor='val_loss')
                         ]

model.fit(train_ds, validation_data=valid_ds , epochs=50 , callbacks=tensorboard_callbacks)

Epoch 1/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5282 - loss: 0.6641 - val_accuracy: 0.7860 - val_loss: 0.4590
Epoch 2/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7936 - loss: 0.4207 - val_accuracy: 0.8342 - val_loss: 0.3688
Epoch 3/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8599 - loss: 0.3185 - val_accuracy: 0.8442 - val_loss: 0.3541
Epoch 4/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8874 - loss: 0.2676 - val_accuracy: 0.8480 - val_loss: 0.3567
Epoch 5/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9076 - loss: 0.2341 - val_accuracy: 0.8428 - val_loss: 0.3718
Epoch 6/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9191 - loss: 0.2103 - val_accuracy: 0.8428 - val_loss: 0.3906
Epoch 7/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9310 - loss: 0.1859 - val_accuracy: 0.8364 - val_loss: 0.4123
Epoch 8/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9408 - loss: 0.1665 - val_accuracy: 0.

In [132]:
%load_ext tensorboard

In [134]:
%tensorboard --logdir my_logs/

Reusing TensorBoard on port 6007 (pid 3970), started 0:01:01 ago. (Use '!kill 3970' to kill it.)

In [137]:
model.get_weights()

[array([[-0.0148323 ,  0.00039671, -0.00739485, ...,  0.01767139,
         -0.02954   ,  0.00295945],
        [-0.01629905, -0.04476994,  0.01218414, ...,  0.03683555,
         -0.01622739, -0.01802848],
        [ 0.02233992,  0.01110129, -0.02978406, ..., -0.01409856,
         -0.00653744, -0.01702018],
        ...,
        [ 0.15417409,  0.02183502, -0.13483599, ..., -0.03213798,
          0.08678889, -0.08537907],
        [ 0.17847452,  0.08058877, -0.15529509, ..., -0.13777556,
          0.06250089, -0.09210343],
        [ 0.02077168,  0.08728888, -0.08879454, ..., -0.09695212,
          0.1407871 , -0.09895734]], dtype=float32),
 array([[-0.3248671 ,  2.5016818 , -1.7128868 , -2.1903372 ,  2.206936  ,
          1.846447  , -2.295708  ,  2.6319044 , -1.4894768 ,  1.9458405 ,
         -2.5153909 , -2.2002606 , -2.3559897 , -0.10990733, -1.7354534 ,
          1.9327366 ],
        [-0.31096733,  2.346235  , -1.7330129 , -1.6590712 ,  2.4341052 ,
          2.4364088 , -2.0215805 ,  2.2

In [138]:
model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_36 (Embedding)        │ (None, 100, 8)         │        80,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_4      │ (None, 8)              │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 240,485 (939.40 KB)

 Trainable params: 80,161 (313.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 160,324 (626.27 KB)

In [145]:
weights = model.get_layer("embedding_36").get_weights()[0]
vocab = vectorized_layer.get_vocabulary()

In [147]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()